In [184]:
import numpy as np
import pandas as pd
import os
import polars as pl
import matplotlib.pyplot as plt
import glob
import tqdm
#from tqdm.notebook import tqdm

import math
import random

In [185]:
hop_1_dict = {}
hop_1_pairs = [(24, 25), (25, 12), (12, 11), (11, 10), (10, 9),
               (9, 21), (3, 4),(4,3),(21, 5),
               (5, 6), (6, 7), (7, 8), (8, 22), (22, 23),(23,8),
               (2, 1), (1, 17),
               (13, 14), (14, 15), (15, 16),(16,15),
               (17, 18), (18, 19), (19, 20),(20,19)]

for start, end in hop_1_pairs:
    hop_1_dict.setdefault(start, []).append(end)

In [186]:
def get_dis(x,y,z)->float:
    return math.sqrt(x**2+y**2+z**2)
def get_two_dis(x1,y1,z1,x2,y2,z2):
    return math.sqrt((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)
def get_two_angle(x1,y1,z1,x2,y2,z2):
    denominator=(math.sqrt(x1**2+y1**2+z1**2)*(math.sqrt(x2**2+y2**2+z2**2)))
    if denominator==0:
        return 0
    else:
        numenrator=(x1*x2)+(y1*y2)+(z1*z2)
        res=numenrator/denominator
        res2=math.acos(res)*(180/math.pi)
        return res2

In [187]:
def get_dis_eachjoint(movement):
    dis_from_h1=[]
    max_frame=movement.iloc[len(movement)-1,4] #4 ==where column frame is
    for frame in range(1,max_frame+1):
        movement_f=movement.loc[movement['frame'] ==frame] #has 25 rows
        for i in range(1, 26):
            hop_indices = hop_1_dict.get(i)
            if hop_indices is not None and len(hop_indices) > 0:
                x1, y1, z1 = movement.iloc[i][['x', 'y', 'z']]
                x2, y2, z2 = movement.iloc[hop_indices[0]][['x', 'y', 'z']]
                res = get_two_dis(x1, y1, z1, x2, y2, z2)
                dis_from_h1.append(res)
    movement['dis_from_h1']=dis_from_h1
    return movement

def get_angle_eachjoint(movement):
    angle_from_h1=[]
    max_frame=movement.iloc[len(movement)-1,4] #4 ==where column frame is
    for frame in range(1,max_frame+1):
        movement_f=movement.loc[movement['frame'] ==frame] #has 25 rows
        for i in range(1, 26):
            hop_indices = hop_1_dict.get(i)
            if hop_indices is not None and len(hop_indices) > 0:
                x1, y1, z1 = movement.iloc[i][['x', 'y', 'z']]
                x2, y2, z2 = movement.iloc[hop_indices[0]][['x', 'y', 'z']]
                res = get_two_angle(x1, y1, z1, x2, y2, z2)
                angle_from_h1.append(res)
    movement['angle_from_h1']=angle_from_h1
    return movement

In [188]:

all_=glob.glob("/Users/kunkerdthaisong/ipu/SampleSkeleton/*.npy",recursive=True)

In [189]:
def to_table(path):#gen_feature
    VIDEO_ = np.load(path, mmap_mode=None, allow_pickle=True)

    shape0, shape1, shape2 = VIDEO_.tolist()['skel_body0'].shape
    movement1 = VIDEO_.tolist()['skel_body0'].reshape(1, shape0, shape1, shape2, 1)
    pos_x,pos_y,pos_z=[],[],[]
    joint=[]
    frames=[]
    f=0
    for frame in movement1[0]:#1 frame (25,3,1) , each movement specific frames
        j=0
        f=f+1
        for x,y,z in frame: #position
            pos_x.append(x[0])
            pos_y.append(y[0])
            pos_z.append(z[0])
            j=j+1
            joint.append(j)
            frames.append(f)
    df_movement1=pd.DataFrame({'x':pos_x,'y':pos_y,'z':pos_z})
    df_movement1['joint']=joint
    df_movement1['frame']=frames
    df_movement1['dis_from_00']=df_movement1.apply(lambda x:get_dis(x.x,x.y,x.z),axis=1)
    x=get_dis_eachjoint(df_movement1)
    y=get_angle_eachjoint(x)
    return y

def to_spectogram(y,save_to):
    data_matrix = y[['x', 'y', 'z', 'dis_from_00', 'dis_from_h1','angle_from_h1']].values.T
    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(data_matrix, cmap='viridis', aspect='auto')
    ax.axis('off')
    dpi = 56
    fig.set_size_inches(448 / dpi, 448 / dpi)  
    fig.savefig(save_to, bbox_inches='tight', pad_inches=0, dpi=dpi)

In [190]:
result

,x,y,z,joint,frame,dis_from_00,dis_from_h1,angle_from_h1
0,0.244869,-0.319706,2.583407,1,1,2.614606,0.642175,13.425974
1,0.229226,0.001406,2.560505,2,1,2.570746,0.315708,7.054169
2,0.213368,0.314537,2.523498,3,1,2.551960,0.331900,7.018092
3,0.264004,0.448448,2.526233,4,1,2.579275,0.331900,7.018092
4,0.077273,0.185574,2.447580,5,1,2.455821,0.257683,6.109904
...,...,...,...,...,...,...,...,...
1645,-0.339183,0.494376,3.139579,21,66,3.196312,0.321745,5.652582
1646,-0.539324,-0.118782,3.187375,22,66,3.234863,0.519260,7.622511
1647,-0.559193,-0.046397,3.159667,23,66,3.209103,0.365881,5.437171
1648,-0.182754,-0.093160,3.139978,24,66,3.146671,0.397694,4.782155


In [191]:
res.iloc[8,8].split("/")[5]

'S003C001P019R002A005.skeleton.npy'

In [192]:

count=0
for i in all_[:2]:
    if count==0:
        res=to_table(i)
        res["file_path"]=[i]*len(res)
    else:
        res1=to_table(i)
        res1["file_path"]=[i]*len(res1)
        res=pd.concat([res,res1])
    count+=1


res

,x,y,z,joint,frame,dis_from_00,dis_from_h1,angle_from_h1,file_path
0,0.244869,-0.319706,2.583407,1,1,2.614606,0.642175,13.425974,/Users/kunkerdthaisong/ipu/SampleSkeleton/S003...
1,0.229226,0.001406,2.560505,2,1,2.570746,0.315708,7.054169,/Users/kunkerdthaisong/ipu/SampleSkeleton/S003...
2,0.213368,0.314537,2.523498,3,1,2.551960,0.331900,7.018092,/Users/kunkerdthaisong/ipu/SampleSkeleton/S003...
3,0.264004,0.448448,2.526233,4,1,2.579275,0.331900,7.018092,/Users/kunkerdthaisong/ipu/SampleSkeleton/S003...
4,0.077273,0.185574,2.447580,5,1,2.455821,0.257683,6.109904,/Users/kunkerdthaisong/ipu/SampleSkeleton/S003...
...,...,...,...,...,...,...,...,...,...
1595,-0.178312,-0.455592,4.002778,21,64,4.032566,0.270153,3.706407,/Users/kunkerdthaisong/ipu/SampleSkeleton/S008...
1596,-0.216316,-0.237826,3.847288,22,64,3.860697,0.438576,6.234500,/Users/kunkerdthaisong/ipu/SampleSkeleton/S008...
1597,-0.211821,-0.315387,3.855286,23,64,3.873960,0.506285,7.218220,/Users/kunkerdthaisong/ipu/SampleSkeleton/S008...
1598,0.044683,-0.981177,3.922552,24,64,4.043652,0.175481,2.431150,/Users/kunkerdthaisong/ipu/SampleSkeleton/S008...


In [194]:
for at,path in tqdm.notebook.tqdm(enumerate(all_),total=len(all_)):
    if at==0:
        result =to_table(path)
    else:
        result2=to_table(path)
        result=pd.concat([result,result2])


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [139]:
result

,x,y,z,joint,frame,dis_from_00,dis_from_h1,angle_from_h1
0,-0.073315,-0.140262,3.736558,1,1,3.739908,0.608695,9.275582
1,-0.070611,0.147910,3.742532,2,1,3.746119,0.282491,4.307542
2,-0.068862,0.430304,3.735337,3,1,3.760671,0.312200,4.600868
3,-0.065369,0.577198,3.753318,4,1,3.798003,0.312200,4.600868
4,-0.211819,0.306925,3.698784,5,1,3.717536,0.217292,3.305945
...,...,...,...,...,...,...,...,...
1670,-0.058877,0.004731,4.176713,21,67,4.177131,0.306659,4.766343
1671,0.059225,-0.373178,3.785649,22,67,3.804459,0.229410,3.652713
1672,-0.000412,-0.371240,3.774667,23,67,3.792879,0.517122,7.864042
1673,-0.089701,-0.362137,3.740066,24,67,3.758628,0.179412,1.940702
